In [1]:
from pyalink.alink import *
resetEnv()
useLocalEnv(1, config=None)



Use one of the following command to start using pyalink:
使用以下一条命令来开始使用 pyalink：
 - useLocalEnv(parallelism, flinkHome=None, config=None)
 - useRemoteEnv(host, port, parallelism, flinkHome=None, localIp="localhost", config=None)
Call resetEnv() to reset environment and switch to another.
使用 resetEnv() 来重置运行环境，并切换到另一个。

JVM listening on 127.0.0.1:60610


JavaObject id=o6

# 手写数字识别
- 使用Softmax 训练模型
- 使用模型预测
- 评估预测结果

# 准备数据

In [2]:
URL = "http://pai-cj.cn-hangzhou.oss.aliyun-inc.com/csv/mnist_dense.csv"
SCHEMA_STR = "label bigint,bitmap string"
mnist_data = CsvSourceBatchOp() \
    .setFilePath(URL) \
    .setSchemaStr(SCHEMA_STR)\
    .setFieldDelimiter(";")
spliter = SplitBatchOp().setFraction(0.8)
train = spliter.linkFrom(mnist_data)
test = spliter.getSideOutput(0)

# 训练 + 预测 + 评估

In [ ]:
softmax = Softmax().setVectorCol("bitmap").setLabelCol("label") \
        .setPredictionCol("pred").setPredictionDetailCol("detail") \
        .setEpsilon(0.0001).setMaxIter(200)
model = softmax.fit(train)
res = model.transform(test)

evaluation = EvalMultiClassBatchOp().setLabelCol("label").setPredictionCol("pred")
metrics = evaluation.linkFrom(res).collectMetrics()

# 打印结果

In [ ]:
print("ConfusionMatrix:", metrics.getConfusionMatrix())
print("LabelArray:", metrics.getLabelArray())
print("LogLoss:", metrics.getLogLoss())
print("TotalSamples:", metrics.getTotalSamples())
print("ActualLabelProportion:", metrics.getActualLabelProportion())
print("ActualLabelFrequency:", metrics.getActualLabelFrequency())
print("Accuracy:", metrics.getAccuracy())
print("Kappa:", metrics.getKappa())